In [2]:
%load_ext sql

#### Connecting to the data warehouse and attaching the database to pull information from the tables. We must be very careful to not delete database tables.

In [3]:
%%sql

sqlite:///CourseDataWarehouseDemo.db
    
attach "CourseData.db" as db1

Done.


[]

#### Using this block of code to make sure of attributes in each table.

In [4]:
%%sql

PRAGMA table_info(COURSE_MEETINGS);

 * sqlite:///CourseDataWarehouseDemo.db
Done.


cid,name,type,notnull,dflt_value,pk
0,CMID,INTEGER,0,None,1
1,LID,INTEGER,1,None,0
2,Term,TEXT,1,None,0
3,CRN,INTEGER,1,None,0
4,Day,TEXT,1,None,0
5,Start,TEXT,1,None,0
6,End,TEXT,1,None,0


#### Dropping tables if need be.

In [1]:
DROP TABLE IF EXISTS INSTRUCTOR_WH;
DROP TABLE IF EXISTS LOCATION_WH;
DROP TABLE IF EXISTS MEETINGS_WH;
DROP TABLE IF EXISTS TOTAL_COURSES_WH;
DROP TABLE IF EXISTS COURSES_WH;

SyntaxError: invalid syntax (<ipython-input-1-e199966188b1>, line 1)

### Creating the tables for our Data Warehouse. There are only 5 tables, with one fact table.

In [5]:
%%sql

CREATE TABLE INSTRUCTOR_WH (
    IID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
(sqlite3.OperationalError) table INSTRUCTOR_WH already exists [SQL: 'CREATE TABLE INSTRUCTOR_WH (\n    IID INTEGER PRIMARY KEY,\n    Name TEXT NOT NULL\n);'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [86]:
%%sql

CREATE TABLE LOCATION_WH (
    LID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]

In [87]:
%%sql

CREATE TABLE MEETINGS_WH (
    MID INTEGER PRIMARY KEY,
    Meetings TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]

In [88]:
%%sql

CREATE TABLE TOTAL_COURSES_WH (
    CRN INTEGER NOT NULL,
    Tot_Courses INTEGER NOT NULL);

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]

In [89]:
%%sql

CREATE TABLE COURSES_WH (
    COID INTEGER PRIMARY KEY,
    CatalogID INTEGER NOT NULL,
    CRN INTEGER,
    Title TEXT NOT NULL,
    LID INTEGER NOT NULL,
    MID INTEGER NOT NULL,
    Section TEXT,
    IID INTEGER NOT NULL,
    Cap TEXT,
    Act TEXT,
    Rem TEXT,
    Tot_Courses INT,
    Credits VARCHAR NOT NULL,
    Prereqs TEXT,
    FOREIGN KEY (LID) REFERENCES LOCATION(LID),
    FOREIGN KEY (MID) REFERENCES MEETINGS(MID),
    FOREIGN KEY (IID) REFERENCES INSTRUCTOR(IID)
    )

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]

#### Inserting data into the tables for the data warehouse to do some queries on and for user use.

In [90]:
%%sql

INSERT INTO INSTRUCTOR_WH (Name)
    SELECT DISTINCT Name
    FROM db1.INSTRUCTOR;

 * sqlite:///CourseDataWarehouseDemo.db
1105 rows affected.


[]

In [91]:
%%sql

INSERT INTO LOCATION_WH (Location)
    SELECT DISTINCT Location
    FROM db1.LOCATION;

 * sqlite:///CourseDataWarehouseDemo.db
979 rows affected.


[]

In [92]:
%%sql

INSERT INTO MEETINGS_WH (Meetings)
    SELECT DISTINCT meetings
    FROM db1.COURSE_OFFERINGS;

 * sqlite:///CourseDataWarehouseDemo.db
12621 rows affected.


[]

In [93]:
%%sql

INSERT INTO TOTAL_COURSES_WH (CRN, Tot_Courses)
    SELECT DISTINCT CRN, COUNT(DISTINCT CRN)
    FROM COURSE_MEETINGS
    GROUP BY CRN;

 * sqlite:///CourseDataWarehouseDemo.db
11482 rows affected.


[]

In [94]:
%%sql

INSERT INTO COURSES_WH (CRN, CatalogID, Title, Section, Cap, Act, Rem, Credits, MID, IID, LID, Tot_Courses)
SELECT DISTINCT CRN, CatalogID, Title, Section, Cap, Act, Rem, Credits, MID, IID, LID, Tot_Courses
FROM COURSE_OFFERINGS co
JOIN MEETINGS_WH mwh ON mwh.Meetings=co.Meetings
JOIN INSTRUCTOR_WH USING (IID)
JOIN COURSE_MEETINGS cm USING (CRN)
JOIN LOCATION_WH lwh USING (LID)
JOIN TOTAL_COURSES_WH tc USING (CRN);

 * sqlite:///CourseDataWarehouseDemo.db
20899 rows affected.


[]

#### Vacuuming the crumbs.

In [6]:
%%sql

vacuum;

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]